# Emotion Recognition Project

In [7]:
# importing from my drive to git
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive
!git clone https://github.com/marius2347/emotion-recognition.git


Mounted at /content/drive
/content/drive/MyDrive
Cloning into 'emotion-recognition'...
remote: Enumerating objects: 34081, done.
remote: Total 34081 (delta 0), reused 0 (delta 0), pack-reused 34081 (from 3)
Receiving objects: 100% (34081/34081), 57.65 MiB | 16.07 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (35889/35889), done.


In [8]:
print("hello")

hello
